In [ ]:
import math, sympy
from fpylll import *

import  random
def rand_prime(bitsize):
    return sympy.randprime(2**(bitsize-1), 2**(bitsize))                        
def base_primes(num_primes):
    primes = list(sympy.primerange(a=2, b=10 ** 5))
    return primes[:num_primes]
def int2bitstring(value, size = None):
    bits = "{:b}".format(value)
    if size:
        bits = '0'*(size-len(bits)) + bits
    return bits    
def M_sizes(matrix):
    res_matrix = IntegerMatrix(matrix)
    nrows = matrix.nrows
    ncols = matrix.ncols
    for i in range(nrows):
        for j in range(ncols):
            res_matrix[i, j] = len(int2bitstring(matrix[i, j]))
    return res_matrix
def equations2reduction(nlin_eqs = 1, dim = 2):
    L = IntegerMatrix(nlin_eqs+dim, dim)
    for i in range(nlin_eqs):
        for j in range(dim):
            L[i, j] = random.randint(0, M)
    for j in range(dim):
        L[nlin_eqs+j, j] = M
    print(L)
    LLL.reduction(L)
    print(L)
    return M_sizes(L)

## Rows count vs reduction

In [10]:
bit_size = 20
M = 2**bit_size

for nlin_eqs in [1,2,3]:
    dim = nlin_eqs + 1
    print(f"num_of equations={nlin_eqs}, num_variables={dim}, \nElement sizes:")
    print(equations2reduction(nlin_eqs = nlin_eqs, dim = dim))

num_of equations=1, num_variables=2, 
Element sizes:
[ 1024329  259206 ]
[ 1048576       0 ]
[       0 1048576 ]
[    0     0 ]
[ -533  -270 ]
[ 1037 -1442 ]
[  1  1 ]
[ 11 10 ]
[ 11 12 ]
num_of equations=2, num_variables=3, 
Element sizes:
[  893724  145861  422558 ]
[   97995  218027  264272 ]
[ 1048576       0       0 ]
[       0 1048576       0 ]
[       0       0 1048576 ]
[    0   0   0 ]
[    0   0   0 ]
[    3 -13 -80 ]
[  -91  71  12 ]
[ -101 -68  -2 ]
[ 1 1 1 ]
[ 1 1 1 ]
[ 2 5 8 ]
[ 8 7 4 ]
[ 8 8 3 ]
num_of equations=3, num_variables=4, 
Element sizes:
[  283993  223189  797632  678874 ]
[  454147  238036   59518  477699 ]
[  890159  288495   68049  112522 ]
[ 1048576       0       0       0 ]
[       0 1048576       0       0 ]
[       0       0 1048576       0 ]
[       0       0       0 1048576 ]
[   0   0   0   0 ]
[   0   0   0   0 ]
[   0   0   0   0 ]
[  21 -10 -11  -9 ]
[  -3 -18   7 -25 ]
[ -17  -2 -28   9 ]
[   0 -35  21  13 ]
[ 1 1 1 1 ]
[ 1 1 1 1 ]
[ 1 1 1 1 ]
[ 5

## Univariate Coppersmith (Howgram-Graham variant)
How large $A$ must be to find $x$ assuming:
 $$(xM+A)(yM+B) = N = p.q$$



In [229]:
bit_size = 100
known_bits = 55
p = rand_prime(bit_size)
q = rand_prime(bit_size)
N = p*q
M = 2**(known_bits)
X = 2**(bit_size-known_bits+1)
A = p % M
x_reference = ( p - A)  // M
assert x_reference < X

$$(xM+A) \equiv 0 \pmod p$$

In [230]:
from sympy import poly, degree, Poly
from sympy.abc import x

fx = poly(x + (A*pow(M, -1, N)) % N)
assert fx.eval(x_reference) % p == 0 

$$(xM+A) \equiv 0 \pmod p$$
$$f(x) = x + c = x+A*M^{-1} \pmod p$$
both polynomials have root $x_0$ i.e. unknow part of $p.$

## Base polynomials
Construct base polynomials $f_k$ such that $f_k(x_0) \equiv 0 \pmod p^m:$
$$f_i = N^{m-i}f^i(x)$$
$$f_j = x^{i}f^m(x)$$


$$
\begin{array}{c|cccccc} 
f_k & 1 & x & x^2 & x^3 & \cdots &  \\
\hline
N^{m-0}f^0(x) & N^m \\
N^{m-1}f^1(x) & *  & N^{m-1}\\
\vdots &  &   \ddots \\
N^{1}f^{m-1}(x) & * & * & N^1\\
x^0f^{m}(x) & *  & *  & *   & 1\\
xf^{m}(x) & * & *  & *   & *  & 1\\
\vdots & *  & *  & *  & *  & &\ddots \\
x^mf^{m}(x) & *  & *  & *   & *  & * & * & 1\\
\end{array}
$$

$$
\begin{array}{c|cccccc} 
f_k & 1 & x & x^2 & x^3 & \cdots &  \\
\hline
N^{m-0}f^0(x) & N^m X\\
N^{m-1}f^1(x) & *  & N^{m-1}X^2\\
\vdots &  &   \ddots \\
N^{1}f^{m-1}(x) & * & * & N^1X^{m-1}\\
x^0f^{m}(x) & *  & *  & *   & X^{m}\\
xf^{m}(x) & * & *  & *   & *  & X^{m+1}\\
\vdots & *  & *  & *  & *  & \ddots \\
x^mf^{m}(x) & *  & *  & *   & *  & * & * X^{2m}\\
\end{array}
$$

In [233]:
def construct_polynomials(fx, N, m, additional):
    polynomials = []
    for i in range(m):
        gi = (fx**i)*(N**(m-i))
        polynomials.append(gi)
    for j in range(additional):
        gi = (fx**m)*x**j
        polynomials.append(gi)
    return polynomials 

def sympoly_to_list(polynomial, dim):
    r = list(reversed(list(map(int, polynomial.as_list(m)))))
    padding = [0]*(dim - len(r))
    return r + padding

def col_weights(M, X, add_weights = True):
    M_copy = IntegerMatrix(M)
    n, m = M_copy.nrows, M.ncols
    for i in range(n):
        for j in range(m):
            if add_weights:
                M_copy[i, j] *= X**j
            else:
                M_copy[i, j] //= X**j
    return M_copy              
def matrix_from_polynomials(polynomials):
    max_degree = max([degree(polynomial) for polynomial in polynomials]) + 1
    matrix = [sympoly_to_list(polynomial, max_degree) for polynomial in polynomials]
    L  = IntegerMatrix.from_matrix(matrix)
    return L
def polynomials_from_matrix(M):
    return [Poly.from_list(reversed(list(row)), x) for row in M]

m = 10
basis_polynomials = construct_polynomials(fx=fx, N = N, m = m, additional=m)
assert [polynomial.eval(x_reference) % p for polynomial in basis_polynomials] == [0]*len(basis_polynomials)   
L  = matrix_from_polynomials(basis_polynomials)
print(M_sizes(L))
L_with_weights = col_weights(L, X, add_weights=True)

[ 1988    1    1    1    1    1    1    1    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1987 1790    1    1    1    1    1    1    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1985 1789 1591    1    1    1    1    1    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1983 1788 1591 1392    1    1    1    1    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1982 1787 1590 1392 1193    1    1    1    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1980 1785 1589 1392 1194  994    1    1    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1978 1784 1588 1391 1194  995  796    1    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1977 1782 1587 1390 1193  995  797  597    1    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1975 1781 1585 1389 1193  995  797  598  398    1    1    1    1    1   1   1   1   1   1 1 ]
[ 1973 1779 1584 1388 1192  995  797  599  400  199    1    1    1    1   1   1   1   1   1 1 ]
[ 1971 1778 1583 1387 1191  994  797  59

In [234]:
LLL.reduction(L_with_weights)
L_removed_weights = col_weights(L_with_weights, X, add_weights=False)
g_polynomials = polynomials_from_matrix(L_removed_weights)

for polynomial in g_polynomials:
    assert polynomial.eval(x_reference) % p == 0
    if polynomial.eval(X) < p**m:
        print(polynomial.eval(x_reference))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-262607899382050987631993942890459077226230957015097435536620038111977059225134646449366177234275121900906882457845920414249167499837093392760008306465814272726552092899968230912044243853185429924147370277356479062151279253813322069268538104313670526692035195036293447080664756569319112172309631372601
